In [88]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer as cv
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import re
import string

In [89]:
df=pd.read_csv("dataset.csv")
df

,crawled_at,verified,reviewed_at,reviewed_by,helpful_count,not_helpful_count,content,raw_content,average_rating,review_heading,product_url,review_page_url,rating
0,"07/02/2021, 01:56:13",False,"November 3, 2019",Greenmachine,0,0,"Great buy, always go with white dark colors me...","<div class=""review-text""><p>Great buy, always ...",NaN,High quality good prices$Bachelor pad,https://www.walmart.com/ip/142064014,https://www.walmart.com/reviews/product/142064014,5.0
1,"07/02/2021, 01:56:17",False,"June 4, 2020",Donna,2,0,Awesome product...love it,"<div class=""review-text""><p>Awesome product......",3.6,NaN,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,5.0
2,"07/02/2021, 01:56:17",True,"May 15, 2021",sherita,0,0,"Love it, easy to put up as well!!!!","<div class=""review-text""><p>Love it, easy to p...",3.6,NaN,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,5.0
3,"07/02/2021, 01:56:17",True,"April 30, 2021",LaLaLady2,0,0,These go up very easily and I love the cordles...,"<div class=""review-text""><p>These go up very e...",3.6,Great blinds at a great price.,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,5.0
4,"07/02/2021, 01:56:17",True,"May 11, 2021",Hope,0,0,"When using the rod to open the blind, the rod ...","<div class=""review-text""><p>When using the rod...",3.6,My mini blind,https://www.walmart.com/ip/203858277,https://www.walmart.com/reviews/product/203858277,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20634,"07/02/2021, 02:19:13",False,"June 5, 2019",Erin M.,0,0,Perfect addition to dining area.,"<div class=""review-text""><p>Perfect addition t...",NaN,Exactly what I expected,https://www.walmart.com/ip/310950951,https://www.walmart.com/reviews/product/310950...,5.0
20635,"07/02/2021, 02:19:13",True,"March 17, 2019",Michael,0,0,"Nice produce , slight problem getting them on ...","<div class=""review-text""><p>Nice produce , sli...",4.4,Tight fit,https://www.walmart.com/ip/21771707,https://www.walmart.com/reviews/product/21771707,4.0
20636,"07/02/2021, 02:19:13",False,"February 7, 2019",Cristina G.,0,0,"Durable, like the coloring a lot","<div class=""review-text""><p>Durable, like the ...",NaN,"Durable, like the coloring a",https://www.walmart.com/ip/310950951,https://www.walmart.com/reviews/product/310950...,5.0
20637,"07/02/2021, 02:19:13",True,"July 17, 2015",Linda,1,0,I love the valance.,"<div class=""review-text""><p>I love the valance...",4.4,Great product!,https://www.walmart.com/ip/21771707,https://www.walmart.com/reviews/product/21771707,5.0


In [90]:
df.columns
df=df[["content","rating"]]

In [91]:
df["rating"]=df["rating"].fillna(df["rating"].mode()[0])


<ipython-input-91-3896d0850e4e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rating"]=df["rating"].fillna(df["rating"].mode()[0])


In [92]:
df=df.dropna()
df

,content,rating
0,"Great buy, always go with white dark colors me...",5.0
1,Awesome product...love it,5.0
2,"Love it, easy to put up as well!!!!",5.0
3,These go up very easily and I love the cordles...,5.0
4,"When using the rod to open the blind, the rod ...",4.0
...,...,...
20634,Perfect addition to dining area.,5.0
20635,"Nice produce , slight problem getting them on ...",4.0
20636,"Durable, like the coloring a lot",5.0
20637,I love the valance.,5.0


In [122]:
def sentiment(n):
    return 1 if n > 4 else 0
df['sentiment'] = df['rating'].apply(sentiment)

df.groupby(["sentiment"]).count()

,content,rating
sentiment,,
0,5542,5542
1,10666,10666


In [94]:
def cleantext(txt):
    txt=str(txt)
    txt=txt.lower()
    txt = re.sub("[,.*!_]","",txt)
    txt = re.sub("\n","",txt)
    txt = re.sub("\t","",txt)
    txt = re.sub("\w*\d\w*","",txt)
    txt = re.sub("^\s+","",txt)
    txt=re.sub("[%s]"%re.escape(string.punctuation),"",txt)
    txt=re.sub("[''""``]","",txt)
    return txt

cle= lambda x:cleantext(x)
df["content"]=pd.DataFrame(df.content.apply(cle))    
x=df.content
y=df["sentiment"]
df["content"]

0        great buy always go with white dark colors mel...
1                                   awesome productlove it
2                           love it easy to put up as well
3        these go up very easily and i love the cordles...
4        when using the rod to open the blind the rod g...
                               ...                        
20634                      perfect addition to dining area
20635    nice produce  slight problem getting them on a...
20636                      durable like the coloring a lot
20637                                   i love the valance
20638    bought this rug for my husbands officeit looks...
Name: content, Length: 16208, dtype: object

In [95]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [96]:
vec=cv()

In [97]:
vec.fit(x)
vec.get_feature_names()
x_train=vec.transform(x_train)
a=pd.DataFrame(x_train.toarray(), columns=vec.get_feature_names())
# a.to_csv("ab.csv")
a

,aaaaaah,aaaah,aaaahs,aaaawesomegood,aahs,abc,abcs,abd,ability,able,...,zippered,zippers,zippy,zips,ziptied,zoom,zucchini,¹ï,útil,œhotâ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
x_test=vec.transform(x_test)

In [99]:
model=LogisticRegression()
model2=KNeighborsClassifier()
model1=DecisionTreeClassifier()

In [100]:
model.fit(x_train,y_train)

c:\python\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [101]:
model.score(x_test,y_test)
# accuracy_score(x_test,y_test)

0.8282329713721619

In [102]:
pre=model.predict(x_test)
model.score(x_test,y_test)
confusion_matrix(y_test,pre)
model.predict(x_test)

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [126]:
stra=[" buy great product"]
rev=vec.transform(stra)


In [127]:
model.predict(rev)

array([1], dtype=int64)

In [123]:
x_train.toarray()

array([[ 898,  410],
       [ 286, 2458]], dtype=int64)